# Refatorando o dataset para facilitar a sua análise

## Imports

In [1]:
import pandas as pd

import ast

from difflib import SequenceMatcher

## Funções

In [2]:
def convert_k_to_int(linha):
    if 'k' in linha:
        linha = linha.replace("k", "")
        linha = float(linha) * 1000
    else:
        linha = float(linha)
    return linha

def remove_comma(linha):
    linha = str(linha).replace(",", "")
    return linha

## Refatorando

In [3]:
# Dataset original
data_original = pd.read_csv('Github_data.txt')

# Dataset refatorado
data_refatorado = data_original.copy()

In [4]:
data_original

,Unnamed: 0.1,Unnamed: 0,topic,name,user,star,fork,watch,issue,pull_requests,...,topic_tag,discription_text,discription_url,commits,branches,packages,releases,contributers,License,url
0,0,0,Data-Science,keras,keras-team,47.9k,18.1k,2.1k,"2,940",87,...,"['deep-learning', 'tensorflow', 'neural-networ...",Deep Learning for humans,http://keras.io/,"5,342",6,0,0,49,View license,https://github.com/keras-team/keras
1,1,1,Data-Science,scikit-learn,scikit-learn,40.3k,19.6k,2.2k,"1,505",764,...,"['machine-learning', 'python', 'statistics', '...",scikit-learn: machine learning in Python,https://scikit-learn.org,"25,405",21,0,0,108,View license,https://github.com/scikit-learn/scikit-learn
2,2,2,Data-Science,PythonDataScienceHandbook,jakevdp,23.1k,9.9k,1.5k,65,67,...,"['scikit-learn', 'numpy', 'python', 'jupyter-n...",Python Data Science Handbook: full text in Jup...,http://jakevdp.github.io/PythonDataSc…,234,5,0,0,0,View license,https://github.com/jakevdp/PythonDataScienceHa...
3,3,3,Data-Science,Probabilistic-Programming-and-Bayesian-Methods...,CamDavidsonPilon,21k,6.6k,1.4k,127,38,...,"['bayesian-methods', 'pymc', 'mathematical-ana...","aka ""Bayesian Methods for Hackers"": An introdu...",http://camdavidsonpilon.github.io/Pro…,"1,079",3,0,0,0,MIT,https://github.com/CamDavidsonPilon/Probabilis...
4,4,4,Data-Science,Data-Science--Cheat-Sheet,abhat222,18.4k,8.2k,1.5k,7,0,...,[],Cheat Sheets,NaN,363,3,0,0,0,Fetching contributors,https://github.com/abhat222/Data-Science--Chea...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,295,95,sensor,react-native-sensor-manager,kprimice,199,53,7,19,5,...,[],Native sensors access for react-native,NaN,45,1,0,0,0,Fetching contributors,https://github.com/kprimice/react-native-senso...
1496,296,96,sensor,weatherstation,androidthings,199,72,31,4,0,...,"['android-things', 'google-cloud-pubsub']",Sensor-based peripheral sample using Android T...,NaN,64,2,0,0,0,Apache-2.0,https://github.com/androidthings/weatherstation
1497,297,97,sensor,OctoPrint-Enclosure,vitormhenrique,198,77,32,63,2,...,"['temperature-sensor', 'enclosure', 'raspberry...",OctoPrint Enclosure Plugin,NaN,415,3,0,0,38,GPL-3.0,https://github.com/vitormhenrique/OctoPrint-En...
1498,298,98,sensor,SensorDashboard,pocmo,197,110,25,0,0,...,[],A sensor dashboard for Android plotting all se...,NaN,103,2,0,0,0,Apache-2.0,https://github.com/pocmo/SensorDashboard


In [5]:
# Trocando o "k"(mil) para int(1000) na coluna "star"
data_refatorado['star'] = data_refatorado['star'].apply(convert_k_to_int)

# Tirando "," na coluna "commits"
data_refatorado['commits'] = data_refatorado['commits'].apply(remove_comma)

In [6]:
# Trocando os nomes repetidos por um "novo" nome, impedindo que haja nomes repetidos no dataset

nomes = []
nomes_repitidos = {}

for i, name in enumerate(data_refatorado['name']):
    name = name.lower()
    if name not in nomes:
        nomes.append(name)
        nomes_repitidos[name] = 1
    else:
        nomes_repitidos[name] = nomes_repitidos[name] + 1
        nomes.append(f"{name}_{nomes_repitidos[name]}")
        data_refatorado.at[i, 'name'] = f"{name}_{nomes_repitidos[name]}"

In [7]:
# Para os repositórios com topic_tag vazio, adicionamos o próprio tópico a lista

for i, value in enumerate(data_refatorado["topic_tag"]):
    if value == "[]":
        data_refatorado.at[i, "topic_tag"] = "['" + data_refatorado.at[i,"topic"] + "']"

In [8]:
# Topicos escritos similarmente se tornam iguais

dicio={} # {letra: [topicos que começam com a letra]}
contador = 0
for index, row in data_refatorado.iterrows():
    cell_value = row["topic_tag"].lower() # em str
    list_result = ast.literal_eval(cell_value) # em list
    
    new_topic_tag = []
    for topico in list_result:
        estado = True
        first_letter = topico[0]
        if first_letter not in dicio.keys():
            dicio[first_letter] = [topico]
        else:
            for topico_dicio in dicio[first_letter]:
                if SequenceMatcher(None, topico, topico_dicio).ratio() >= 0.95:
                    topico = topico_dicio
                    estado = False
            if estado == True:
                dicio[first_letter].append(topico)
        new_topic_tag.append(topico)
                
    data_refatorado.at[index, "topic_tag"] = str(new_topic_tag)
    
    contador+=1
    print(contador, end = "\r")

## Mostrando o dataset refatorado

In [10]:
data_refatorado

,Unnamed: 0.1,Unnamed: 0,topic,name,user,star,fork,watch,issue,pull_requests,...,topic_tag,discription_text,discription_url,commits,branches,packages,releases,contributers,License,url
0,0,0,Data-Science,keras,keras-team,47900.0,18.1k,2.1k,"2,940",87,...,"['deep-learning', 'tensorflow', 'neural-networ...",Deep Learning for humans,http://keras.io/,5342,6,0,0,49,View license,https://github.com/keras-team/keras
1,1,1,Data-Science,scikit-learn,scikit-learn,40300.0,19.6k,2.2k,"1,505",764,...,"['machine-learning', 'python', 'statistics', '...",scikit-learn: machine learning in Python,https://scikit-learn.org,25405,21,0,0,108,View license,https://github.com/scikit-learn/scikit-learn
2,2,2,Data-Science,PythonDataScienceHandbook,jakevdp,23100.0,9.9k,1.5k,65,67,...,"['scikit-learn', 'numpy', 'python', 'jupyter-n...",Python Data Science Handbook: full text in Jup...,http://jakevdp.github.io/PythonDataSc…,234,5,0,0,0,View license,https://github.com/jakevdp/PythonDataScienceHa...
3,3,3,Data-Science,Probabilistic-Programming-and-Bayesian-Methods...,CamDavidsonPilon,21000.0,6.6k,1.4k,127,38,...,"['bayesian-methods', 'pymc', 'mathematical-ana...","aka ""Bayesian Methods for Hackers"": An introdu...",http://camdavidsonpilon.github.io/Pro…,1079,3,0,0,0,MIT,https://github.com/CamDavidsonPilon/Probabilis...
4,4,4,Data-Science,Data-Science--Cheat-Sheet,abhat222,18400.0,8.2k,1.5k,7,0,...,['data-science'],Cheat Sheets,NaN,363,3,0,0,0,Fetching contributors,https://github.com/abhat222/Data-Science--Chea...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,295,95,sensor,react-native-sensor-manager,kprimice,199.0,53,7,19,5,...,['sensor'],Native sensors access for react-native,NaN,45,1,0,0,0,Fetching contributors,https://github.com/kprimice/react-native-senso...
1496,296,96,sensor,weatherstation,androidthings,199.0,72,31,4,0,...,"['android-things', 'google-cloud-pubsub']",Sensor-based peripheral sample using Android T...,NaN,64,2,0,0,0,Apache-2.0,https://github.com/androidthings/weatherstation
1497,297,97,sensor,OctoPrint-Enclosure,vitormhenrique,198.0,77,32,63,2,...,"['temperature-sensor', 'enclosure', 'raspberry...",OctoPrint Enclosure Plugin,NaN,415,3,0,0,38,GPL-3.0,https://github.com/vitormhenrique/OctoPrint-En...
1498,298,98,sensor,SensorDashboard,pocmo,197.0,110,25,0,0,...,['sensor'],A sensor dashboard for Android plotting all se...,NaN,103,2,0,0,0,Apache-2.0,https://github.com/pocmo/SensorDashboard


In [11]:
# Mostrando apenas as colunas que iremos analisar
data_refatorado.loc[:, ['topic', 'name', 'star', 'topic_tag', 'commits']]

,topic,name,star,topic_tag,commits
0,Data-Science,keras,47900.0,"['deep-learning', 'tensorflow', 'neural-networ...",5342
1,Data-Science,scikit-learn,40300.0,"['machine-learning', 'python', 'statistics', '...",25405
2,Data-Science,PythonDataScienceHandbook,23100.0,"['scikit-learn', 'numpy', 'python', 'jupyter-n...",234
3,Data-Science,Probabilistic-Programming-and-Bayesian-Methods...,21000.0,"['bayesian-methods', 'pymc', 'mathematical-ana...",1079
4,Data-Science,Data-Science--Cheat-Sheet,18400.0,['data-science'],363
...,...,...,...,...,...
1495,sensor,react-native-sensor-manager,199.0,['sensor'],45
1496,sensor,weatherstation,199.0,"['android-things', 'google-cloud-pubsub']",64
1497,sensor,OctoPrint-Enclosure,198.0,"['temperature-sensor', 'enclosure', 'raspberry...",415
1498,sensor,SensorDashboard,197.0,['sensor'],103


## Salvando o dataset refatorado

In [12]:
data_refatorado.to_csv('Github_data_refatorado.csv', index=False)